In [1]:
import pandas as pd
from seeq import spy
from seeq.spy.assets import Asset, ItemGroup
pd.options.display.max_columns = None


In [2]:
WORKBOOK_NAME = None
DATASOURCE_NAME = 'F1 Telemetry Playground'
SESSION_CONDITION_NAME = 'F1 Sessions'


In [3]:
def get_f1_items():
    scoped_workbook_id = None
    if WORKBOOK_NAME != None:
        scoped_workbook = spy.search({'Name':WORKBOOK_NAME, 'Type':'Workbook'})
        scoped_workbook_id = scoped_workbook['ID'][0]
    
    telemetry_items = spy.search({'Name':'*','Datasource Name':DATASOURCE_NAME, 'Scoped To':scoped_workbook_id})
    return telemetry_items

In [4]:
class F1_Car(Asset):
    
    @Asset.Attribute()
    def Sectors(self, metadata):
        return metadata[metadata['Name'].str.endswith('Sectors')]
    
    @Asset.Attribute()
    def X_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('X')]

    @Asset.Attribute()
    def Y_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('Y')]

    @Asset.Attribute()
    def Z_pos(self, metadata):
        return metadata[metadata['Name'].str.endswith('Z')]

    @Asset.Attribute()
    def RPM(self, metadata):
        return metadata[metadata['Name'].str.endswith('RPM')]

    @Asset.Attribute()
    def Throttle(self, metadata):
        return metadata[metadata['Name'].str.endswith('Throttle')]

    @Asset.Attribute()
    def Brake(self, metadata):
        return metadata[metadata['Name'].str.endswith('Brake')]

    @Asset.Attribute()
    def Gear(self, metadata):
        return metadata[metadata['Name'].str.endswith('GearNo')]

    #@Asset.Attribute()
    #def TimestampPOS(self, metadata):
    #    return metadata[metadata['Name'].str.endswith('timestamp_pos')]

    @Asset.Attribute()
    def Speed(self, metadata):
        return metadata[metadata['Name'].str.endswith('Speed')]

    @Asset.Attribute()
    def LapNumber(self, metadata):
        return metadata[metadata['Name'].str.endswith('LapNo')]

    @Asset.Attribute()
    def DRS(self, metadata):
        return metadata[metadata['Name'].str.endswith('DRS')]

    #@Asset.Attribute()
    #def SessionKeyPOS(self, metadata):
    #    return metadata[metadata['Name'].str.endswith('SessionKey_pos')]

    @Asset.Attribute()
    def CarStatus(self, metadata):
        return metadata[metadata['Name'].str.endswith('CarStatus')]

    @Asset.Attribute()
    def TrackStatus(self, metadata):
        return metadata[metadata['Name'].str.endswith('TrackStatus')]

    @Asset.Attribute()
    def Compound(self, metadata):
        return metadata[metadata['Name'].str.endswith('Compound')]

    @Asset.Attribute()
    def Distance(self, metadata):
        return metadata[metadata['Name'].str.endswith('Distance')]

    @Asset.Attribute()
    def TireAge(self, metadata):
        return metadata[metadata['Name'].str.endswith('TyreAge')]

#####  Calculations

    @Asset.Attribute()
    def Laps(self, metadata):
        return {
            'Type': 'Condition',
            
            # This formula will create a capsule for every change in the lap number value
            'Formula': "$h.toCondition().renameProperty('value','lap number')",
            
            'Formula Parameters': {
                # We can reference the base class lap number
                '$h': self.LapNumber(),
            }
        }
            
    @Asset.Attribute()
    def Sector_1(self, metadata):
        return {
            'Type': 'Condition',
            
            # This formula will create a new condition that filters out only sector 1
            'Formula': "$condition.keep('Sector', isEqualTo(1))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_2(self, metadata):
        return {
            'Type': 'Condition',
            
            'Formula': "$condition.keep('Sector', isEqualTo(2))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_3(self, metadata):
        return {
            'Type': 'Condition',
            
            'Formula': "$condition.keep('Sector', isEqualTo(3))",
            
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }
    

In [5]:
class F1_Car(Asset):

    @Asset.Attribute()
    def Sectors(self, metadata):
        return metadata[metadata['Name'].str.endswith('Sectors')]

    @Asset.Attribute()
    def X_pos(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('X')]
            }
        }

    @Asset.Attribute()
    def Y_pos(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('Y')]
            }
        }

    @Asset.Attribute()
    def Z_pos(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('Z')]
            }
        }

    @Asset.Attribute()
    def RPM(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('RPM')]
            }
        }

    @Asset.Attribute()
    def Throttle(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('Throttle')]
            }
        }

    @Asset.Attribute()
    def Brake(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('Brake')]
            }
        }

    @Asset.Attribute()
    def Gear(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('GearNo')]
            }
        }

    #@Asset.Attribute()
    #def TimestampPOS(self, metadata):
    #    return metadata[metadata['Name'].str.endswith('timestamp_pos')]

    @Asset.Attribute()
    def Speed(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('Speed')]
            }
        }

    @Asset.Attribute()
    def LapNumber(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('LapNo')]
            }
        }

    @Asset.Attribute()
    def DRS(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('DRS')]
            }
        }

    #@Asset.Attribute()
    #def SessionKeyPOS(self, metadata):
    #    return metadata[metadata['Name'].str.endswith('SessionKey_pos')]

    @Asset.Attribute()
    def CarStatus(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('CarStatus')]
            }
        }

    @Asset.Attribute()
    def TrackStatus(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('TrackStatus')]
            }
        }

    @Asset.Attribute()
    def Compound(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('Compound')]
            }
        }

    @Asset.Attribute()
    def Distance(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('Distance')]
            }
        }

    @Asset.Attribute()
    def TireAge(self, metadata):
        return {
            'Type': 'Signal',
            'Formula': "$signal.setMaxInterpolation(15min)",
            'Formula Parameters': {
                '$signal': metadata[metadata['Name'].str.endswith('TyreAge')]
            }
        }

#####  Calculations

    @Asset.Attribute()
    def Laps(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': "$h.toCondition().renameProperty('value','lap number')",
            'Formula Parameters': {
                '$h': self.LapNumber(),
            }
        }

    @Asset.Attribute()
    def Sector_1(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': "$condition.keep('Sector', isEqualTo(1))",
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_2(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': "$condition.keep('Sector', isEqualTo(2))",
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }

    @Asset.Attribute()
    def Sector_3(self, metadata):
        return {
            'Type': 'Condition',
            'Formula': "$condition.keep('Sector', isEqualTo(3))",
            'Formula Parameters': {
                '$condition': self.Sectors(),
            }
        }


In [6]:
telemetry_items = get_f1_items()
telemetry_items[['Season', 'Team', 'Car Number', 'Tag Name']] = telemetry_items['Name'].str.split('.', expand=True)
telemetry_items['Build Asset'] = telemetry_items['Car Number'] 
telemetry_items ['Build Path'] = 'F1 Telemetry >> ' + telemetry_items['Season'] + ' >> ' + telemetry_items['Team']
telemetry_items.to_csv('telemetryitems.csv')

,Name,Datasource Name,Scoped To,Time,Count,Pages,Result
0,*,F1 Telemetry Playground,None,00:00:00.72,630,1,Success


In [7]:
all_items = get_f1_items()
all_items[0:20]

,Name,Datasource Name,Scoped To,Time,Count,Pages,Result
0,*,F1 Telemetry Playground,None,00:00:00.60,630,1,Success


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,0F098A32-8E4F-6470-9523-5B64557E18F6,2025.Alpine.10.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
1,0F098A65-6378-EE80-B463-2408D0BA8475,2025.Alpine.7.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
2,0F098A33-4855-7590-B909-9EF8A168CFD8,2025.Aston Martin.14.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
3,0F098A33-FC6F-64E0-BCED-78A6CF03B113,2025.Aston Martin.18.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
4,0F098A33-9C4D-EEE0-AF93-CF6F35556266,2025.Ferrari.16.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
5,0F098A58-BD9A-FDD0-BEBA-4AD993663651,2025.Ferrari.44.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
6,0F098A54-CF05-7510-B18C-449B76B3DEA8,2025.Haas.31.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
7,0F098A67-CC51-E810-8538-72B467AC6448,2025.Haas.87.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
8,0F098A50-BDA0-EA30-8595-37054D0F675A,2025.Kick Sauber.27.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False
9,0F098A5B-A5C1-ECB0-903A-5BDD1E3C54C8,2025.Kick Sauber.5.Sectors,NaN,StoredCondition,NaN,F1 Telemetry Playground,False


In [8]:
build_df = spy.assets.build(F1_Car, telemetry_items)
build_df['Maximum Interpolation'] = '15 min'

,Build Path,Build Asset,Build Template,Build Result
0,F1 Telemetry >> 2025 >> Alpine,10,F1_Car,Success
1,F1 Telemetry >> 2025 >> Alpine,7,F1_Car,Success
2,F1 Telemetry >> 2025 >> Aston Martin,14,F1_Car,Success
3,F1 Telemetry >> 2025 >> Aston Martin,18,F1_Car,Success
4,F1 Telemetry >> 2025 >> Ferrari,16,F1_Car,Success
5,F1 Telemetry >> 2025 >> Ferrari,44,F1_Car,Success
6,F1 Telemetry >> 2025 >> Haas,31,F1_Car,Success
7,F1 Telemetry >> 2025 >> Haas,87,F1_Car,Success
8,F1 Telemetry >> 2025 >> Kick Sauber,27,F1_Car,Success
9,F1 Telemetry >> 2025 >> Kick Sauber,5,F1_Car,Success


In [9]:

spy.push(metadata=build_df, workbook=WORKBOOK_NAME)

,Signal,Scalar,Condition,Threshold Metric,Display,Display Template,Asset,Datafile,Relationship,Overall,Time
Items pushed,0,0,0,0,0,0,0,0,0,0,0



SPy Error: Error processing signal: The parameter had an empty dataframe

Error found at line 1 in cell 9.


Button(description='Click to show stack trace', layout=Layout(height='auto', width='auto'), style=ButtonStyle(…